# Решение #4 - Итеративная кластеризация

## Содержание
* [Описание решения](#description)
* [Подготовительные действия](#preprocess)
* [Решение](#solution)
* [Результаты](#result)

## Описание решения <a id='description'></a>

#### Основой решения является алгоритм кластеризации К-средних
#### Порядок действий:
1) Предобработка строк
2) Векторизация подсчетом
3) Стандартизация
4) Уменьшение размерности методом главных компонент
5) Применение алгоритма К-средних
6) Получение кластеров строк
7) Повторение шагов 2-6 n раз (n - подбирается экспериментально) для каждой группы рекурсивно
8) Разделение неправильно сгруппированных строк с помощью расстояния Карловского и формирование групп

#### Использованные базовые алгоритмы:
* Метод кластеризации К-средних
* Растояние Карловского

## Подготовительные действия <a id='preprocess'></a>

### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

import warnings
warnings.filterwarnings("ignore")

df = pd.read_json('./../data/sample.json')

### Структура датасета

In [2]:
df.head()

,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


### Предобработка строк

In [3]:
df['text_preprocess'] = df['text'].apply(load_dataset.preprocess_text)
strings = list(df['text_preprocess'])
strings

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей пбмолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал влера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

## Решение <a id='solution'></a>

### Функция группировки строк

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from typing import List

# Группировка строк
def clusterization(strings:List[str])->List[List[str]]:
    n_clusters = 2 # Экспериментально подобранное значение количества кластеров
    
    # Если в списке меньше n_clusters, то их нельзя поделить на n_clusters, поэтому возвращаем исходный список 
    if len(set(strings)) <= n_clusters:
        return [strings]
        
    # Векторизация подсчетом
    Vectorizer = CountVectorizer()
    vector = Vectorizer.fit_transform(strings)
    strings_vectorized = vector.toarray()

    # Стандартизация векторов 
    strings_vectorized = StandardScaler().fit_transform(strings_vectorized)

    # Уменьшение размерности с помощью метода главных компонент
    strings_vectorized_nD = PCA(n_components=len(strings_vectorized)//2,svd_solver='full').fit_transform(strings_vectorized)

    # Кластеризация методом К-средних
    model = KMeans.KMeans(n_clusters=n_clusters).fit(strings_vectorized_nD)

    # Группировка строк по получившимся кластерам
    clusters = {}
    for i in range(len(model.labels_)):
        if model.labels_[i] not in clusters:
            clusters[model.labels_[i]] = []
        clusters[model.labels_[i]].append(strings[i])

    return list(clusters.values())

### Итеративная кластеризация

In [5]:
coef = 8 # Экспериментально подобранный коэффициент

# Проходим несколько раз функцией группировки строк для получения более мелких групп
clusters = clusterization(strings)
for iter in tqdm(range(len(strings)//coef)):
    for i,cluster in enumerate(clusters):
        clusters[i] = clusterization(cluster)
    clusters = [item for cluster in clusters for item in cluster];

  0%|          | 0/51 [00:00<?, ?it/s]

### Формирование групп

In [6]:
groups = []
threshold = 0.89

# Дополнительная проверка строк на рерайт и формирование конечных групп
for cluster in tqdm(clusters):
    # Группы длиной 1 не подвергаются дополнительной проверке
    if len(cluster) == 1:
        groups.append(cluster)
        
    # Дополнительная проверка с помощью расстояния Карловского
    if len(cluster) == 2:
        if karlovskiy_distance.is_rewrite_karlovskiy_distance(cluster[0],cluster[1],threshold):
            groups.append(cluster)  
        else:
            groups.append([cluster[0]])
            groups.append([cluster[1]])
            
    if len(cluster) > 2:
        used = [False] * len(cluster)
        for i in range(len(cluster)):
            if used[i]:
                continue
            for j in range(i+1,len(cluster)):
                if used[j]:
                    continue
                if karlovskiy_distance.is_rewrite_karlovskiy_distance(cluster[i],cluster[j],threshold):
                    used[i] = True
                    used[j] = True
                    groups.append([cluster[i],cluster[j]])
        for i in range(len(cluster)):
            if not used[i]:
                groups.append([cluster[i]])

  0%|          | 0/122 [00:00<?, ?it/s]

### Преобразование в строки исходного датасета

In [7]:
for i in range(len(groups)):
    for j in range(len(groups[i])):
        groups[i][j] = df[df['text_preprocess'] == groups[i][j]]['text'].to_numpy()[0]

## Результаты <a id='result'></a>

### Все найденные группы

In [8]:
print('Количество групп = ',len(groups))

Количество групп =  279


In [9]:
groups

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?', 'Почему он так со мной поступает?'],
 ['Почему они с нами так поступают?'],
 ['Почему они так с ним поступают?'],
 ['Почему она так с ней поступает?'],
 ['Почему она с ним так поступает?', 'Почему она так с ним поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['Том никогда туда больше не вернётся.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я беру всё с собой.'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Тому было не с кем играть.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Я сделал всё правильно?'],
 ['Он мне сделал комплимент.', 'Он сделал мне комплимент.'],
 ['По-моему, я это сделал неправильно.'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['У меня сейчас есть немного ден

### Строки, которые имеют более 1 рерайта

In [10]:
groups3 = [group for group in groups if len(group) > 2]

In [11]:
print('Количество групп = ',len(groups3))

Количество групп =  0


In [12]:
groups3

[]

### Строки, которые имеют ровно 1 рерайт

In [13]:
groups2 = [group for group in groups if len(group) == 2]

In [14]:
print('Количество групп = ',len(groups2))

Количество групп =  133


In [15]:
groups2

[['Почему она так со мной поступает?', 'Почему он так со мной поступает?'],
 ['Почему она с ним так поступает?', 'Почему она так с ним поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Он мне сделал комплимент.', 'Он сделал мне комплимент.'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.'],
 ['Кто тебе это всё сказал?', 'Кто тебе это всё сказал?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Я никогда не забуду выражение его лица.',
  'Я никогда не забуду его выражение лица.'],
 ['Том никогда тебя больше не увидит.', 'Том никогда больше тебя не увидит.'],
 ['Я никогда больше не увижу Тома.', 'Я больше никогда не увижу Тома

### Строки, которые не имеют рерайта

In [16]:
groups1 = [group for group in groups if len(group) == 1]

In [17]:
print('Количество групп = ',len(groups1))

Количество групп =  146


In [18]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему они с нами так поступают?'],
 ['Почему они так с ним поступают?'],
 ['Почему она так с ней поступает?'],
 ['Том никогда туда больше не вернётся.'],
 ['Я беру всё с собой.'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было не с кем играть.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Я сделал всё правильно?'],
 ['По-моему, я это сделал неправильно.'],
 ['У меня сейчас есть немного денег.'],
 ['Том не хочет никого видеть.'],
 ['Он очень этого хочет.'],
 ['Том сказал, что ничего не хочет есть.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Тому это тоже не нравится.'],
 ['Я тебе уже всё сказал.'],
 ['Что именно сказал тебе Том?'],
 ['Том больше мне не доверяет.'],
 ['Я только что его доделал.'],
 ['Я только что его исправил.'],
 ['Я так его любила.'],
 ['Мы больше не хотим ждать.'],
 ['Он больше не может ждать.'],
 ['Том больше не хотел об этом думать.'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Я не живу в Бостоне.'],
 ['

### Сохранение результата

In [19]:
save_groups.save_groups(groups,"../output/solution4-result.json")